In [ ]:
from pyglotaran_model_language.top_level.clp_relation import ClpRelation

In [ ]:
ClpRelation.__annotations__["source"]

In [ ]:
dir(ClpRelation)

In [12]:
from pydantic import Field, Extra
from pyglotaran_model_language.model_item_base import ModelItemBase
from pyglotaran_model_language.label_types import ParameterLabel
import abc
from typing import Literal, cast, Any

class Foo(ModelItemBase):
    type: Literal["foo","bar"] = Field("foo", const=True, description="Required")
    foo: list[ParameterLabel] = Field(__hidden__="__foo__")
    
    @property
    def parameter_labels(self) -> set[ParameterLabel]:
        return set(self.foo)
    
    class Config:
        @staticmethod
        def schema_extra(schema: dict[str, Any], model: type["ModelItemBase"]) -> None:
            for key, val in schema.get("properties", {}).items():
                if key == "type":
                    val.pop("default", None)
                    val.pop("const", None)
                    required_fields = schema.get("required", [])
                    schema["required"] = ["type", *required_fields]

    
# foo = Foo(foo=["1","2"])
foo = Foo(type="foo",foo=[ParameterLabel("1")])
foo
from rich import print as pprint
pprint(Foo.schema())


{
    'title': 'Foo',
    'description': 'Base class for model items ensuring that they have properties returning all the used labels.',
    'type': 'object',
    'properties': {
        'type': {'title': 'Type', 'description': 'Required', 'enum': ['foo', 'bar'], 'type': 'string'},
        'foo': {'title': 'Foo', '__hidden__': '__foo__', 'type': 'array', 'items': {'type': 'string'}}
    },
    'required': ['type', 'foo']
}

In [ ]:
Foo.__fields__["type"].type_

In [ ]:
Foo.__fields__["foo"].annotation in (ParameterLabel, list[ParameterLabel])

In [ ]:
type(list[ParameterLabel])

In [ ]:
Foo.__fields__["type"].annotation

In [ ]:
Foo.__annotations__["foo"]

In [ ]:
ParameterLabel == Foo.__annotations__["foo"]

In [ ]:
Foo.__annotations__["foo"]

In [ ]:
"ParameterLabel" in repr(Foo.__annotations__.values())

In [ ]:
Foo.__class__.__qualname__

In [ ]:
import inspect
from pydantic import BaseModel

inspect.signature(Foo.__init_subclass__)

In [ ]:
Foo.schema()

# Jörn stuff

In [ ]:
"""This module contains the item classes and helper functions."""
import contextlib
from dataclasses import MISSING
from functools import cache
from inspect import getmro
from inspect import isclass
from types import NoneType
from types import UnionType
from typing import ClassVar
from typing import Generator
from typing import Type
from typing import TypeAlias
from typing import TypeVar
from typing import Union
from typing import get_args
from typing import get_origin

from pydantic import BaseModel
from pydantic import Field

from glotaran.parameter import Parameter

ItemT = TypeVar("ItemT", bound="Item")
ModelItemT = TypeVar("ModelItemT", bound="ModelItem")

ParameterType: TypeAlias = Parameter | str
ModelItemType: TypeAlias = ModelItemT | str  # type:ignore[operator]

__DATACLASS_ARGS = {"kw_only": True, "config": {"arbitrary_types_allowed": True}}
META_ALIAS = "__glotaran_alias__"
META_VALIDATOR = "__glotaran_validator__"


class Item(BaseModel):
    """A baseclass for items."""

    class Config:
        arbitrary_types_allowed = True
        
        


class TypedItem(Item):
    """An item with a type."""

    type: str
    __item_types__: ClassVar[dict[str, Type]]

    def __init_subclass__(cls: type["TypedItem"]):
        """Create an item from a class.

        Parameters
        ----------
        cls: type[ItemT]
            The class.

        Returns
        -------
        type[ItemT]
        """
        if cls.__qualname__ == "ModelItemTyped":
            return
        parent = getmro(cls)[1]
        if parent in (TypedItem, ModelItemTyped):
            assert issubclass(cls, TypedItem)
            cls.__item_types__ = {}
        elif issubclass(cls, TypedItem):
            cls._register_item_class()

    @classmethod
    def _register_item_class(cls):
        """Register a class as type."""
        item_type = cls.get_item_type()
        if item_type is not MISSING:
            cls.__item_types__[item_type] = cls

    @classmethod
    def get_item_type(cls) -> str:
        """Get the type string.

        Returns
        -------
        str
        """
        return cls.__fields__["type"].default

    @classmethod
    def get_item_types(cls) -> list[str]:
        """Get all type strings.

        Returns
        -------
        list[str]
        """
        return list(cls.__item_types__.keys())

    @classmethod
    def get_item_type_class(cls, item_type: str) -> Type:
        """Get the type for a type string.

        Parameters
        ----------
        item_type: str
            The type string.
        Returns
        -------
        Type
        """
        return cls.__item_types__[item_type]


class ModelItem(Item):
    """An item with a label.

    Attributes
    ----------
    label : str
        The label of the model item.
    """

    label: str


class ModelItemTyped(TypedItem, ModelItem):
    """A model item with a type."""


class MockModelItem(ModelItem):
    pass
        
class MockItem(Item):
    cscalar: int
    cscalar_option: int | None
    clist: list[int]
    clist_option: list[int] | None
    cdict: dict[str, int]
    cdict_option: dict[str, int] | None
    iscalar: ModelItemType[MockModelItem]
    iscalar_option: ModelItemType[MockModelItem] | None
    ilist: list[ModelItemType[MockModelItem]]
    ilist_option: list[ModelItemType[MockModelItem]] | None
    idict: dict[str, ModelItemType[MockModelItem]]
    idict_option: dict[str, ModelItemType[MockModelItem]] | None
    pscalar: ParameterType
    pscalar_option: ParameterType | None
    plist: list[ParameterType]
    plist_option: list[ParameterType] | None
    pdict: dict[str, ParameterType]
    pdict_option: dict[str, ParameterType] | None


@cache
def get_structure_and_type_from_field(field: Field) -> tuple[None | list | dict, type]:
    """Get the structure and type from a field.

    Parameters
    ----------
    field: Field
        The field.

    Returns
    -------
    tuple[None | list | dict, type]:
        The structure and type as atuple.
    """
    print(field.name)
    print(field.type_)
    definition = strip_option_type_from_definition(field.type_)
    print(definition)
    structure, definition = strip_structure_type_from_definition(definition)
    definition = strip_option_type_from_definition(definition, strip_type=str)
    return structure, definition

In [ ]:
get_structure_and_type_from_field(MockItem.__fields__["clist"])

In [ ]:
MockItem.__fields__["clist"].type_

In [ ]:
MockItem.__fields__["clist"]

In [ ]:
MockItem.__fields__["clist"].annotation

In [ ]:
MockItem.__fields__["clist"].annotation == list[int]

In [ ]:
from pydantic.fields import Undefined